In [3]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
df = pd.read_csv(r"D:\Recommendation System\anime.csv")

In [7]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [9]:
#checking for null values

df.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [12]:
# Filling the null values

df['genre'].fillna('unknown', inplace =True)


In [13]:
df['type'].fillna('unknown', inplace =True)
df['rating'].fillna('not rated', inplace =True)

C:\Users\PC\AppData\Local\Temp\ipykernel_9084\202824991.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'not rated' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['rating'].fillna('not rated', inplace =True)


In [14]:
df.isna().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [28]:
# Convert float columns to strings
df['name'] = df['name'].astype(str)
df['genre'] = df['genre'].astype(str)
df['rating'] = df['rating'].astype(str)

In [72]:
df.reset_index(inplace=True)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     12294 non-null  int64 
 1   anime_id  12294 non-null  int32 
 2   name      12294 non-null  object
 3   genre     12294 non-null  object
 4   type      12294 non-null  object
 5   episodes  12294 non-null  object
 6   rating    12294 non-null  object
 7   members   12294 non-null  int64 
dtypes: int32(1), int64(2), object(5)
memory usage: 720.5+ KB


In [75]:
df

,index,anime_id,name,genre,type,episodes,rating,members
0,0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...,...
12289,12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [74]:
df.shape

(12294, 8)

In [31]:
# selecting the relevant features for recommendation

selected_features = ['name','genre','rating']
selected_features

['name', 'genre', 'rating']

In [32]:
# combining all the  selected features

combining_feature = df['name']+''+df['genre']+''+df['rating']
combining_feature

0        Kimi no Na wa.Drama, Romance, School, Supernat...
1        Fullmetal Alchemist: BrotherhoodAction, Advent...
2        Gintama°Action, Comedy, Historical, Parody, Sa...
3                          Steins;GateSci-Fi, Thriller9.17
4        Gintama&#039;Action, Comedy, Historical, Parod...
                               ...                        
12289    Toushindai My Lover: Minami tai Mecha-MinamiHe...
12290                                Under WorldHentai4.28
12291             Violence Gekiga David no HoshiHentai4.88
12292    Violence Gekiga Shin David no Hoshi: Inma Dens...
12293           Yasuji no Pornorama: Yacchimae!!Hentai5.46
Length: 12294, dtype: object

In [33]:
# converting the text data to feature vectors

vectorizing = TfidfVectorizer()

In [34]:
feature_vectors = vectorizing.fit_transform(combining_feature)


<12294x17925 sparse matrix of type '<class 'numpy.float64'>'
	with 90207 stored elements in Compressed Sparse Row format>

In [35]:
print(feature_vectors)

  (0, 251)	0.3514153029877354
  (0, 15210)	0.5782092334836741
  (0, 13578)	0.22631788221599736
  (0, 13056)	0.21378198891561448
  (0, 3639)	0.2148386961619564
  (0, 16862)	0.29607497171521896
  (0, 10506)	0.3477794941886842
  (0, 11061)	0.14916589739733918
  (0, 7999)	0.4101728279342529
  (1, 198)	0.2832845368969268
  (1, 14430)	0.4164104465518756
  (1, 9895)	0.22798275736635748
  (1, 9212)	0.20413346105958285
  (1, 4188)	0.15299048503267226
  (1, 479)	0.17541402976064396
  (1, 1832)	0.4856478690344826
  (1, 680)	0.41052786199426605
  (1, 4489)	0.405265710794507
  (1, 3639)	0.17292240662370847
  (2, 194)	0.30097362661195204
  (2, 4247)	0.20506609091319952
  (2, 13595)	0.16725642262254556
  (2, 13409)	0.3423684113928725
  (2, 11948)	0.2987457717306961
  (2, 6037)	0.25228154705453126
  :	:
  (12289, 69)	0.2600908118454791
  (12290, 17144)	0.6954809389210207
  (12290, 16545)	0.5963282180568098
  (12290, 201)	0.4008726979313077
  (12291, 6206)	0.5086553234832882
  (12291, 3132)	0.472396558

In [36]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [37]:
print(similarity)

[[1.         0.03715042 0.         ... 0.01875772 0.01575334 0.01858004]
 [0.03715042 1.         0.20591276 ... 0.         0.         0.        ]
 [0.         0.20591276 1.         ... 0.         0.         0.        ]
 ...
 [0.01875772 0.         0.         ... 1.         0.54992867 0.01566346]
 [0.01575334 0.         0.         ... 0.54992867 1.         0.01315468]
 [0.01858004 0.         0.         ... 0.01566346 0.01315468 1.        ]]


In [38]:
similarity.shape

(12294, 12294)

In [57]:
anime_name = input("Enter the anime name: ")
print(anime_name)

Enter the anime name: Gintama°
Gintama°


In [58]:
list_of_tiltle = df['name'].tolist()
list_of_tiltle

['Kimi no Na wa.',
 'Fullmetal Alchemist: Brotherhood',
 'Gintama°',
 'Steins;Gate',
 'Gintama&#039;',
 'Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou',
 'Hunter x Hunter (2011)',
 'Ginga Eiyuu Densetsu',
 'Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare',
 'Gintama&#039;: Enchousen',
 'Clannad: After Story',
 'Koe no Katachi',
 'Gintama',
 'Code Geass: Hangyaku no Lelouch R2',
 'Haikyuu!! Second Season',
 'Sen to Chihiro no Kamikakushi',
 'Shigatsu wa Kimi no Uso',
 'Mushishi Zoku Shou 2nd Season',
 'Ookami Kodomo no Ame to Yuki',
 'Code Geass: Hangyaku no Lelouch',
 'Hajime no Ippo',
 'Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen',
 'Cowboy Bebop',
 'One Punch Man',
 'Mononoke Hime',
 'Suzumiya Haruhi no Shoushitsu',
 'Monogatari Series: Second Season',
 'Mushishi Zoku Shou',
 'Mushishi',
 'Tengen Toppa Gurren Lagann',
 'Great Teacher Onizuka',
 'Natsume Yuujinchou Go',
 'Hajime no Ippo: New Challenger',
 'Mushishi Zoku Shou: Suzu no Shizuku',
 'Natsume Yuuji

In [59]:
# finding the close match for the movie name given by the user

find_the_match = difflib.get_close_matches(anime_name,list_of_tiltle)
close_match = find_the_match[0]
close_match

'Gintama°'

In [76]:
# finding the index of the movie with title

index_of_the_anime = df[df.name == close_match]['index'].values[0]
index_of_the_anime                                            

2

In [77]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_anime]))
similarity_score

[(0, 0.0),
 (1, 0.20591275568423467),
 (2, 1.0),
 (3, 0.04080161647783948),
 (4, 0.8594628114389199),
 (5, 0.0),
 (6, 0.06976240471990755),
 (7, 0.06355681196272206),
 (8, 0.5409446072644313),
 (9, 0.7041572867249659),
 (10, 0.0),
 (11, 0.23916249525057884),
 (12, 0.5984078460963543),
 (13, 0.05277186164903397),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.04913356662137613),
 (18, 0.0),
 (19, 0.05750356680082387),
 (20, 0.0),
 (21, 0.04146764118444405),
 (22, 0.09641291956460002),
 (23, 0.16554037844943906),
 (24, 0.0),
 (25, 0.06078908319170023),
 (26, 0.0),
 (27, 0.05614418932316903),
 (28, 0.06409957656977015),
 (29, 0.04845412259057412),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.04524748668464744),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.17229672430347784),
 (42, 0.027808634276522075),
 (43, 0.037295850697224756),
 (44, 0.0),
 (45, 0.0),
 (46, 0.0),
 (47, 0.0),
 (48, 0.05367534678375234),
 (49, 0.0),
 (50, 0.0),
 (51, 0.06854599

In [78]:
len(similarity_score)

12294

In [82]:
# sorting the movies based on their similarity score

sorted_similarity_anime = sorted(similarity_score, key = lambda x:x[1], reverse = True)
sorted_similarity_anime

[(2, 1.0),
 (4, 0.8594628114389199),
 (9, 0.7041572867249659),
 (10896, 0.6002146202429091),
 (12, 0.5984078460963543),
 (8, 0.5409446072644313),
 (63, 0.48707058609714615),
 (216, 0.4590859954623664),
 (306, 0.45862511441956144),
 (65, 0.44977896910799325),
 (361, 0.3958600076151174),
 (171, 0.3558388210011748),
 (104, 0.31765708058935554),
 (316, 0.2984051187619171),
 (380, 0.2859620161920577),
 (1295, 0.2680267981822402),
 (2631, 0.26485533988793464),
 (4301, 0.255604159113204),
 (1603, 0.24801610498487298),
 (11, 0.23916249525057884),
 (6125, 0.23875092917889557),
 (6049, 0.2362430584704203),
 (11055, 0.2318419383374381),
 (3246, 0.23049227941652806),
 (4507, 0.2301404388782514),
 (622, 0.228653812019587),
 (5922, 0.2218262089820152),
 (7362, 0.21939813026092425),
 (8440, 0.21792535074582736),
 (6433, 0.2173058061829903),
 (4912, 0.21668994620851628),
 (3469, 0.21662159320680016),
 (10927, 0.21600428893216814),
 (5554, 0.21506616723847033),
 (10657, 0.21303715785997998),
 (6048, 0.

In [83]:
# print the name of similar movies based on the index

print("Movies suggested for you : \n  ")

i = 1

for movie in sorted_similarity_anime:
  index = movie[0]
  title_from_index = df[df.index==index]['name'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 
  
1 . Gintama°
2 . Gintama&#039;
3 . Gintama&#039;: Enchousen
4 . Gintama (2017)
5 . Gintama
6 . Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare
7 . Gintama: Yorinuki Gintama-san on Theater 2D
8 . Gintama: Shinyaku Benizakura-hen
9 . Gintama: Jump Festa 2014 Special
10 . Gintama Movie: Shinyaku Benizakura-hen
11 . Gintama: Jump Festa 2015 Special
12 . Gintama: Shiroyasha Koutan
13 . Samurai Champloo
14 . Gintama: Dai Hanseikai
15 . Gintama: Nanigoto mo Saiyo ga Kanjin nano de Tasho Senobisuru Kurai ga Choudoyoi
16 . Samurai 7
17 . Yoroiden Samurai Troopers
18 . Tono to Issho
19 . Senyuu.
20 . Koe no Katachi
21 . Watashi no Coffee Samurai: Jihanki-teki na Kareshi
22 . Tono to Issho 1.5
23 . Peace Maker Kurogane (Shinsaku)
24 . Gintama x Mameshiba
25 . Sengoku Otome: Momoiro Paradox
26 . Drifters
27 . D.I.C.E.
28 . Samurai Spirits: Haten Gouma no Shou
29 . Dennou Sentai Voogie&#039;s★Angel Gaiden: Susume! Super★Angels!


In [85]:
anime_name = input("Enter the anime name: ")

list_of_tiltle = df['name'].tolist()

find_the_match = difflib.get_close_matches(anime_name,list_of_tiltle)
close_match = find_the_match[0]

index_of_the_anime = df[df.name == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_anime]))

sorted_similarity_anime = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print("Movies suggested for you : \n  ")

i = 1

for movie in sorted_similarity_anime:
  index = movie[0]
  title_from_index = df[df.index==index]['name'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Enter the anime name: Clannad: After Story
Movies suggested for you : 
  
1 . Clannad: After Story
2 . Clannad Movie
3 . Clannad: After Story - Mou Hitotsu no Sekai, Kyou-hen
4 . Kimi no Na wa.
5 . Sola Specials
6 . Ai Shoujo Pollyanna Story
7 . Colorful (Movie)
8 . Bologee Story
9 . After School
10 . Clannad: Mou Hitotsu no Sekai, Tomoyo-hen
11 . Natsu-iro Kiseki
12 . Life!
13 . After... The Animation
14 . Kirara
15 . Kanon
16 . Air
17 . Sola
18 . Clannad
19 . Kanon Kazahana
20 . Touch
21 . Dororonpa!
22 . Hyakka Ryouran: Samurai After Specials
23 . Kanokon
24 . A-Channel
25 . Fuujin Monogatari
26 . Peeping Life Specials
27 . Kanon (2006)
28 . Pupipo!
29 . Free!
